# Authentications
First, let's authenticate the colab notebook to access both GCP and GEE.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

# Imports
Note that we downgraded tensorflow to `2.1.0`. That's because that's the tensorflow runtime version  that we use to train, eeify, and deploy the model later on. So we want to make sure that this colab notebook is using the same version.


In [ ]:
!pip install tensorflow==2.1.0
import tensorflow as tf
print(tf.__version__)

In [ ]:
import folium

# Config Variables

In [ ]:
# INSERT YOUR PROJECT HERE!
PROJECT = 'GCP project Id'

# INSERT YOUR BUCKET HERE!
BUCKET = 'your bucket'

# Specify names of output locations in Cloud Storage.
JOB_FOLDER = 'your folder'
JOB_DIR = 'gs://' + BUCKET + '/' + JOB_FOLDER + '/trainer'
MODEL_DIR = JOB_DIR + '/model'
LOGS_DIR = JOB_DIR + '/logs'

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = JOB_DIR + '/eeified'

# Pre-computed training and eval data.
DATA_BUCKET = 'bucket from data prep notebook'
FOLDER = 'FOLDER from data prep notebook'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = ['B10', 'B11']
BANDS = opticalBands + thermalBands
RESPONSE = 'cultivated'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 10
BUFFER_SIZE = 3000
OPTIMIZER = 'adam'
LOSS = 'binary_crossentropy'
METRICS = ['binary_accuracy']

# EEification of the Model
Before we can host the model on AI Platform we need to EEify it. The EEification process appends some extra operations to the input and outputs of the model in order to accomdate the interchange format between pixels from Earth Engine (float32) and inputs to AI Platform (base64). (See https://cloud.google.com/ai-platform/prediction/docs/online-predict#binary_data_in_prediction_input for details.)

The EEification process is handled for you using the Earth Engine command `earthengine model prepare`. To use that command, we need to specify the input and output model directories and the name of the input and output nodes in the TensorFlow computation graph. We can do all that programmatically:

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "impervious"}) + "'"

The project needs to be set  before using the model prepare command.

In [ ]:
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

# Model Deployment
I personally prefer doing this via AI Platforms model console but it can be done via command line as well.

In [ ]:
%%writefile config.yaml
autoScaling:
    minNodes: 10

In [ ]:
MODEL_NAME = 'model_name'
VERSION_NAME = 'version_name' 
REGION = 'region_name'

!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --region {REGION} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version 2.1 \
  --python-version 3.7 \
  --config=config.yaml

# Inference using the deployed model
Earth Engine can directly be connected to the trained model for inference using `ee.Model.fromAiPlatformPredictor` command. To connect to the model, we need to know the name and version.

### Inputs
we need to be able to recreate the imagery on which it was trained in order to perform inference.  Specifically, we need to create an array-valued input from the scaled data and use that for input.  (Recall that the new input node is named `array`, which is convenient because the array image has one band, named `array` by default.)  The inputs will be provided as 144x144 patches (`inputTileSize`), at 30-meter resolution (`proj`), but 8 pixels will be thrown out (`inputOverlapSize`) to minimize boundary effects.

### Outputs
The output (which we also need to know), is a single float band named `cultivated`.

In [ ]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(config.opticalBands).gt(0).And(
          image.select(config.opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(config.opticalBands).divide(10000).addBands(
          image.select(config.thermalBands).divide(10).clamp(273.15, 373.15)
            .subtract(273.15).divide(100)).updateMask(mask)

# The image input data is a cloud-masked median composite.
image = l8sr.filterDate(
    '2015-01-01', '2017-12-31').map(maskL8sr).median().select(config.BANDS).float()

# Load the trained model and use it for prediction.
model = ee.Model.fromAiPlatformPredictor(
    projectName = PROJECT,
    modelName = MODEL_NAME,
    version = VERSION_NAME,
    inputTileSize = [144, 144],
    inputOverlapSize = [8, 8],
    proj = ee.Projection('EPSG:4326').atScale(30),
    fixInputProj = True,
    outputBands = {'cultivated': {
        'type': ee.PixelType.float()
      }
    }
)

predictions = model.predictImage(image.toArray())
# Use folium to visualize the input imagery and the predictions.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[36.71, -120.55], zoom_start=20)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
mapid = predictions.getMapId({'min': 0, 'max': 1})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='cultivated',
  ).add_to(map)
map.add_child(folium.LayerControl())
map